In [1]:
DEBUG = False

In [2]:

# import sklearn
# import sklearn.ensemble
# import sklearn.metrics
# import sklearn.feature_extraction
# from sklearn.pipeline import make_pipeline
import pointing_game_util

In [3]:
import pandas as pd

In [4]:
test = pd.read_pickle("./dataset_test.pkl")
test = test # always load the full dataset! (np.random.shuffle(tokenized_sentences)). slice the actual hybrid_documents if debugging!
documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py

In [5]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorGuo,DetectorRadford,]# DetectorDetectGPT]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer
explainer_classes = [LIME_Explainer, SHAP_Explainer]

In [6]:
# keep a copy of the hybrid documents for the assert in the loop
ref_assert_hybrid_documents, _, _ = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=LIME_Explainer(DetectorRadford()).tokenize)


In [7]:
results = []
for detector_class in detector_classes:
    detector = detector_class()
    print("Initialized " + detector.__class__.__name__)
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        print("Initialized " + explainer.__class__.__name__)

        print("Indexing hybrid documents for " + explainer.__class__.__name__)
        hybrid_documents, tokenized_hybrid_documents, GT = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=explainer.tokenize)
        assert (all([a==b for a,b in zip(ref_assert_hybrid_documents,hybrid_documents)])), "(full) Hybrid documents don't match" # tokenized_hybrid_documents differ by design to make the calculation of the pointing game accuracy easier

        if DEBUG:
            hybrid_documents = hybrid_documents[0:10]
            tokenized_hybrid_documents = tokenized_hybrid_documents[0:10]
            GT = GT[0:10]

        print("Predicting hybrid documents")
        predictions_hybrid = detector.predict_label(hybrid_documents)

        print("Generating explanations on hybrid documents and calculating pointing game accuracy")

        
        pointing_game_acc = pointing_game_util.get_pointing_game_acc(hybrid_documents, explainer, predictions_hybrid, GT)

        print("Pointing game accuracy for {} | {}: {}".format(explainer.__class__.__name__, detector.__class__.__name__, pointing_game_acc))
        results.append((explainer.__class__.__name__, detector.__class__.__name__, pointing_game_acc))

        




Initialized DetectorGuo
Initialized LIME_Explainer
Indexing hybrid documents for LIME_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for LIME_Explainer | DetectorGuo: 1.0
Initialized SHAP_Explainer
Indexing hybrid documents for SHAP_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for SHAP_Explainer | DetectorGuo: 0.7137476459510358
Initialized DetectorRadford
Initialized LIME_Explainer
Indexing hybrid documents for LIME_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy
Pointing game accuracy for LIME_Explainer | DetectorRadford: 0.4934086629001883
Initialized SHAP_Explainer
Indexing hybrid documents for SHAP_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy

In [8]:
df = pd.DataFrame(results, columns=["Explanation Method", "Detector", "Pointing Game ACC"])
df

,Explanation Method,Detector,Pointing Game ACC
0,LIME_Explainer,DetectorGuo,1.000000
1,SHAP_Explainer,DetectorGuo,0.713748
2,LIME_Explainer,DetectorRadford,0.493409
3,SHAP_Explainer,DetectorRadford,0.826742


In [9]:
# import shap

In [10]:

# for hybrid_document in hybrid_documents:

#     explainer = LIME_Explainer(detector)
#     explainer.get_explanation_cached(hybrid_document).show_in_notebook()

#     explainer = SHAP_Explainer(detector)
#     shap.text_plot(explainer.get_explanation_cached(hybrid_document))
        